In [1]:
import pandas as pd
import numpy as np
#from sklearn.preprocessing import StandardScaler   #,MinMaxScaler

In [2]:
dir_dados = '../2_dados/'
arq_sumario = dir_dados + 'data_sumario.parquet'
arq_serie = dir_dados + '/data_series/data_{}.npy'

In [3]:
arq_treino_sumario = dir_dados + '/treino_teste/treino_sumario.parquet'
arq_treino = dir_dados + '/treino_teste/treino.npy'
arq_teste_sumario = dir_dados + '/treino_teste/teste_sumario.parquet'
arq_teste = dir_dados + '/treino_teste/teste.npy'

### Abre arquivos

In [4]:
df = pd.read_parquet(arq_sumario)

In [5]:
df.head()

,signal_id,phase,target,arquivo
0,0,0,0,1
1,1,1,0,1
2,2,2,0,1
3,3,0,1,2
4,4,1,1,2


In [6]:
df['target'].value_counts()

0    466
1     35
Name: target, dtype: int64

In [7]:
df[['phase','target']].groupby(['phase','target']).size()

phase  target
0      0         156
       1          11
1      0         156
       1          11
2      0         154
       1          13
dtype: int64

### Preparação de dados

#### Divide em Treino e Teste

In [8]:
classe_neg_idx = df[df['target'] == 0]
classe_pos_idx = df[df['target'] == 1]

In [9]:
treino_neg_prop = int(classe_neg_idx.shape[0] * 0.7)    # 70% da classe negativa
treino_pos_prop = int(classe_pos_idx.shape[0] * 0.7)    # 70% da classe positiva

In [10]:
treino_idx = classe_neg_idx[:treino_neg_prop].append(classe_pos_idx[:treino_pos_prop])
teste_idx = classe_neg_idx[treino_neg_prop:].append(classe_pos_idx[treino_pos_prop:])

In [11]:
print('Treino:')
print(treino_idx['target'].value_counts())

Treino:
0    326
1     24
Name: target, dtype: int64


In [12]:
# Balanceia treino, deixando os negativos na mesma qtd do positivos
treino_idx = treino_idx[-48:]

In [13]:
print('Treino:')
print(treino_idx['target'].value_counts())

Treino:
1    24
0    24
Name: target, dtype: int64


In [14]:
print('Teste:')
print(teste_idx['target'].value_counts())

Teste:
0    140
1     11
Name: target, dtype: int64


In [15]:
def getSerie(df_index):
    dataset = []
    for _, linha in df_index.iterrows():
        serie = np.load(arq_serie.format(linha['arquivo']))
        fase = serie[linha['phase']]
        dataset.append(fase)
    return np.array(dataset)

In [16]:
treino = getSerie(treino_idx)
print('Dimensões de treino:', treino.shape)

Dimensões de treino: (48, 800000)


In [17]:
teste = getSerie(teste_idx)
print('Dimensões de teste:', teste.shape)

Dimensões de teste: (151, 800000)


#### Obtem dados estatísticos das séries
(mínimo, máximo, média, mediana, desvio padrão e percentis 25 e 75)

In [18]:
treino = np.nan_to_num(treino.astype(np.int32), nan=0, posinf=0, neginf=0)
teste = np.nan_to_num(teste.astype(np.int32), nan=0, posinf=0, neginf=0)

In [19]:
#scaler = StandardScaler()   #StandardScaler(), MinMaxScaler()
def getMeanDataset(dataset):
    qtd = 2000
    data = []
    for linha in dataset:
        serie = []
        for i in range(0, len(linha), qtd):
            fatia = linha[i:i + qtd]
            serie.append([np.min(fatia), np.max(fatia), np.mean(fatia), np.percentile(fatia, 25), np.median(fatia), np.percentile(fatia, 75), np.std(fatia)])
        #serie = scaler.fit_transform(serie)
        data.append(serie)
    return np.array(data)

In [20]:
treino = getMeanDataset(treino)
teste = getMeanDataset(teste)

In [21]:
print('Dimesão de treino:', treino.shape)
print('Dimesão de teste:', teste.shape)

Dimesão de treino: (48, 400, 7)
Dimesão de teste: (151, 400, 7)


#### Salva arquivos

In [22]:
treino_idx.to_parquet(arq_treino_sumario)
teste_idx.to_parquet(arq_teste_sumario)

In [23]:
np.save(arq_treino, treino)
np.save(arq_teste, teste)